# IMC-Denoise

# Default setup IMC-Denoise

1- Follow the instructions under the 'Installation' header from here: https://github.com/PENGLU-WashU/IMC_Denoise. In brief, you need to setup a new conda environment and install some packages with specific version numbers, and then clone and install the IMCDenoise package from Github.
2- Run the following command in Anaconda prompt to install a couple of extra packages we will need in the new environment: conda install tqdm pandas seaborn

# Denoising from non-Bodenmiller pipeline sources

This will also work with images from a non-Bodenmiller source. When you define channels, it will search through the source directories for any images matching that name. Therefore, be careful with channels with overlapping names! Eg. CD4 and CD45!

# Imports and functions

In [1]:
import os
from os import listdir
from os.path import isfile, join, abspath, exists
from glob import glob
import tifffile as tp
import pandas as pd
import seaborn as sb
from pathlib import Path
from copy import copy
from tqdm import tqdm
from scipy.stats import zscore
import distutils.dir_util

from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tp

In [2]:
import sys
import keras
import tensorflow as tf
import tensorflow.keras
import pandas as pd
import sklearn as sk
import platform

In [3]:
tensorflow.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
tf.config.run_functions_eagerly(False)

In [5]:
print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: macOS-12.6-arm64-arm-64bit
Tensor Flow Version: 2.10.0
Keras Version: 2.10.0

Python 3.9.13 | packaged by conda-forge | (main, May 27 2022, 17:00:33) 
[Clang 13.0.1 ]
Pandas 1.5.0
Scikit-Learn 1.1.2
GPU is available


In [6]:
from IMC_Denoise.IMC_Denoise_main.DIMR import DIMR
from IMC_Denoise.IMC_Denoise_main.DeepSNF import DeepSNF
from IMC_Denoise.DeepSNF_utils.DeepSNF_DataGenerator import DeepSNF_DataGenerator, load_training_patches

In [7]:
import dill

In [111]:
### These are adapted from functions from IMC_Denoise

def load_single_img(filename):
    
    """
    Loading single image from directory.
    Parameters
    ----------
    filename : The image file name, must end with .tiff.
        DESCRIPTION.
    Returns
    -------
    Img_in : int or float
        Loaded image data.
    """
    if filename.endswith('.tiff') or filename.endswith('.tif'):
        Img_in = tp.imread(filename).astype('float32')
    else:
        raise ValueError('Raw file should end with tiff or tif!')
    if Img_in.ndim != 2:
        raise ValueError('Single image should be 2d!')
    return Img_in

def load_imgs_from_directory(load_directory,channel_name,quiet=False):
    Img_collect = []
    img_folders = glob(join(load_directory, "*", ""))
    Img_file_list=[]

    if not quiet:
        print('Image data loaded from ...\n')
    
    for sub_img_folder in img_folders:
        Img_list = [f for f in listdir(sub_img_folder) if isfile(join(sub_img_folder, f)) & (f.endswith(".tiff") or f.endswith(".tif"))]
        for Img_file in Img_list:
            if channel_name.lower() in Img_file.lower():
                Img_read = load_single_img(sub_img_folder + Img_file)
                
                if not quiet:
                    print(sub_img_folder + Img_file)
                
                Img_file_list.append(Img_file)
                Img_collect.append(Img_read)
                break

    if not quiet:
        print('\n' + 'Image data loaded completed!')
    
    if not Img_collect:
        print(f'No such channel as {channel_name}. Please check the channel name again!')
        return

    return Img_collect, Img_file_list, img_folders



# This function takes the stacked 'ometiffs' which the Bodenmiller pipeline extracts from the MCD files right at the start of the pipeline, then extracts them to individual channels with sensible names

def unstack_tiffs(input_folder = 'tiff_stacks', #The folder with the ometiffs
            unstacked_output_folder = 'tiffs', #The name of the folder where tiffs will be extracted
            use_panel_files=True): #Use panel files created by BM pipeline for each ROI

    global channel_df
    global all_data_channels
    global roi_data
    
    # Make output directories if they don't exist
    input_folder = Path(input_folder)
    output = Path(unstacked_output_folder)
    output.mkdir(exist_ok=True)

    # Setup a blank dataframe ready to add to
    if use_panel_files:
        #roi_data = pd.DataFrame(columns=['panel_filename','channel_name','channel_label','filename','folder','fullstack_path'])
        roi_data = pd.DataFrame(columns=['channel_name','channel_label'])
    # Get a list of all the .tiff files in the input directory
    tiff_files = list(input_folder.rglob('*.tiff'))

    print('Unpacking ROIs...')
    for roi_count,i in enumerate(tqdm(tiff_files)):

        image = tp.imread(str(i))    

        folder_name = os.path.splitext(os.path.basename(i))[0]

        tiff_folder_name = os.path.splitext(os.path.basename(i))[0]    
        output_dir = Path(unstacked_output_folder,tiff_folder_name)
        output_dir.mkdir(exist_ok=True)        

        if use_panel_files:

            panel_filename = os.path.splitext(os.path.splitext(os.path.basename(i))[0])[0] + '.csv'
            panel_path = join(*i.parts[0:-1])
            panel_df = pd.read_csv(join(panel_path, panel_filename))
            panel_df['fullstack_path'] = copy(str(i))       
            panel_df['panel_filename']=panel_filename
            panel_df['folder']=folder_name
            roi_data = pd.concat([roi_data, panel_df], sort=True)

        for channel_count in range(image.shape[0]):

            if use_panel_files:

                panel_df['filename']=copy(str(channel_count)).zfill(2)+"_"+str(roi_count).zfill(2)+"_"+panel_df['channel_name']+"_"+panel_df['channel_label'].astype(str)+".tiff"
                tp.imwrite(join(output_dir, panel_df.loc[channel_count,'filename']), image[channel_count])
            else:
                file_name=copy(str(channel_count)).zfill(2)+"_"+str(roi_count).zfill(2)+".tiff"
                tp.imwrite(join(output_dir, file_name), image[channel_count])        

    if use_panel_files:
        roi_data.to_csv('ROI_data.csv')       
        all_data_channels = roi_data.dropna().channel_label.unique().tolist()
        all_data_channel_names = roi_data.dropna().channel_name.unique().tolist()
        channel_df = pd.DataFrame(list(zip(all_data_channel_names,all_data_channels)), columns = ['channel_name', 'channel_label'])
        channel_df['channel']=channel_df['channel_name'] + "_" + channel_df['channel_label']
        channel_df.to_csv('channels_list.csv')

        blank_channels = roi_data[roi_data.channel_label.isna()].channel_name.unique()
        n = len(blank_channels)
        print(f'The following {n} EMPTY channels were detected, and will be NOT be processed... \n')
        print(roi_data[roi_data.channel_label.isna()].channel_name.unique().tolist())

        n = len(all_data_channels)
        print(f'\nThe following {n} channels were detected, and will be used if process_all_channels=True in the next step... \n')
        print(channel_df['channel'])

### This is for doing QC heatmaps and PCAs to look for outliers

def qc_heatmap(directory='tiffs', #The directory to analyse
                quantile=0.95,
                save=True, 
                process_all_channels=False,
                channels=[],
                normalize=None, #Can be max or zscore
                figsize=(20,10),
                dpi=200,             
                save_dir='qc_images',
                do_PCA=True,
                annotate_PCA=True,
                hide_figures=False):
    
    if not isinstance(channels, list):
        channels=[channels]  

    if process_all_channels:
        channels = channel_df['channel'].tolist()
        
    # Create folder for saving
    save_dir = Path(save_dir)
    save_dir.mkdir(exist_ok=True)
    
    # Create lists to save data into
    channel_list=[]
    roi_list=[]
    img_max_list=[]
    img_mean_list=[]
    img_std_list=[]
    img_q_list=[]

    
    print('Extracting data from images...\n')
    for channel in tqdm(channels):

        Img_collect, Img_file_list, img_folders = load_imgs_from_directory(directory, channel, quiet=True)    

        for img,img_f in zip(Img_collect,img_folders):
            roi = Path(img_f).parts[1]
            img_max=np.max(img)
            img_mean=np.mean(img)
            img_std=np.std(img)
            img_q=np.quantile(img,quantile)

            channel_list.append(copy(channel))
            roi_list.append(copy(roi))
            img_max_list.append(copy(img_max))
            img_mean_list.append(copy(img_mean))
            img_std_list.append(copy(img_std))
            img_q_list.append(copy(img_q))


    results_df = pd.DataFrame(list(zip(channel_list, roi_list, img_max_list, img_mean_list, img_std_list, img_q_list)), columns=['channel','ROI','max','mean', 'std','quantile'])

    print('Plotting results...\n')

    for i in ['max','mean','quantile', 'std','quantile']:
        results_pivot = pd.pivot_table(results_df, index='channel',columns='ROI', values=i)

        if normalize=='max':
            results_pivot = results_pivot.div(results_pivot.max(axis=1), axis=0)
            i = i+'_max_normalised'
        elif normalize=='zscore':
            results_pivot = results_pivot.apply(zscore, axis=0)
            i = i+'_zscore'         

        fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
        ax = sb.heatmap(results_pivot,xticklabels=True, yticklabels=True)
        plt.title(i)

        if save:
            fig.savefig(join(save_dir,f'{directory}_{i}_heatmap.png'))
        
        if hide_figures:
            plt.close()
                        
        if do_PCA:
            import sklearn
            from sklearn.decomposition import PCA

            scaled_summary_data = sklearn.preprocessing.StandardScaler().fit_transform(results_pivot.T)

            pca = PCA(n_components=2)
            embedding = pca.fit_transform(scaled_summary_data)

            #Create the graphs
            fig, ax = plt.subplots(figsize=(10,10))
            ax.scatter(
                embedding[:, 0],
                embedding[:, 1],
                s=15)
            
            if annotate_PCA:
                for loc, txt in zip(embedding,list(range(len(results_pivot.T.index)))):
                    ax.annotate(txt, loc)  
                pd.DataFrame(results_pivot.T.index).to_csv(join(save_dir,'roi_annotations.csv'))                    
            
            fig.gca().set_aspect('equal', 'datalim')
            ax.set_xlabel('PCA1')
            ax.set_ylabel('PCA2')
            plt.title(i)
            
            if save:
                plt.savefig(join(save_dir,f'{directory}_{i}_PCA.png'))
                
            if hide_figures:
                plt.close()

#### Deep SNF batch function            
            
def deep_SNF_batch(raw_directory = "tiffs", #Input folder
                   processed_output_dir = "processed", #Output folder
                   process_all_channels = False,
                   channels = [], 
                   patch_step_size=60,
                   train_epoches = 50, # 50 gets good results in my experience.
                    train_initial_lr = 1e-3, # inital learning rate. The default is 1e-3.
                    train_batch_size = 128, # training batch size. For a GPU with smaller memory, it can be tuned smaller. The default is 256.
                    pixel_mask_percent = 0.2, # percentage of the masked pixels in each patch. The default is 0.2.
                    val_set_percent = 0.15, #percentage of validation set. The default is 0.15.
                    loss_function = "I_divergence", # loss function used. The default is "I_divergence".
                    loss_name = None, # training and validation losses saved here, either .mat or .npz format. If not defined, the losses will not be saved.
                    weights_save_directory = None, # location where 'weights_name' and 'loss_name' saved.
                    # If the value is None, the files will be saved in a sub-directory named "trained_weights" of  the current file folder.
                    is_load_weights = False, # Use the trained model directly. Will not read from saved one.
                    lambda_HF = 3e-6,
                    n_neighbours = 4, # Larger n enables removing more consecutive hot pixels 
                    n_iter = 3, # Iteration number for DIMR
                    window_size = 3): # HF regularization parameter

    # Error catching to make specific channels a list if just one channel given
    if not isinstance(channels, list):
        channels=[channels]  

    # Training settings
    row_step=patch_step_size
    col_step=patch_step_size 

    # Create folders
    processed_output_dir = Path(processed_output_dir)
    processed_output_dir.mkdir(exist_ok=True)

    # Error catching lists
    error_channels=[]
    completed_channels=[]

    if process_all_channels:
        channels = channel_df['channel'].tolist()

    n = len(channels)
    print(f'\nPerforming denoising on the following {n} channels... \n')
    print(channels)
    
        
    for channel_name in tqdm(channels):

        try:

            if 'generated_patches' in globals():
                del globals.generated_patches    

            if not is_load_weights:
                DataGenerator = DeepSNF_DataGenerator(channel_name = channel_name, 
                                                      n_neighbours = n_neighbours, # Larger n enables removing more consecutive hot pixels 
                                                      n_iter = n_iter, # Iteration number for DIMR
                                                      window_size = window_size, # Slide window size. For IMC images, window_size = 3 is fine.
                                                      col_step=col_step,
                                                      row_step=row_step)

                generated_patches = DataGenerator.generate_patches_from_directory(load_directory = raw_directory)
                print('The shape of the generated training set is ' + str(generated_patches.shape) + '.')

            weights_name="weights_"+str(channel_name)+".hdf5"
            
            deepsnf = DeepSNF(train_epoches = train_epoches, 
                              train_learning_rate = train_initial_lr,
                              train_batch_size = train_batch_size,
                              mask_perc_pix = pixel_mask_percent,
                              val_perc = val_set_percent,
                              loss_func = loss_function,
                              weights_name = weights_name,
                              loss_name = loss_name,
                              weights_dir = weights_save_directory, 
                              is_load_weights = is_load_weights,
                              lambda_HF = lambda_HF)

            
            if not is_load_weights:
                print('STARTING TRAINING...')
                # Train the DeepSNF classifier 
                train_loss, val_loss = deepsnf.train(generated_patches)
            else:
                print(f'Using weights file: {weights_name}')

            # Load all images
            Img_collect, Img_file_list, img_folders = load_imgs_from_directory(raw_directory, channel_name)

            # Save resulting images
            for i, img_file_name, folder in zip(Img_collect, Img_file_list, img_folders):

                #Perform both the hot pixel and shot noise 
                Img_DIMR_DeepSNF = deepsnf.perform_IMC_Denoise(i, n_neighbours = n_neighbours, n_iter = n_iter, window_size = window_size)

                #Gets the ROI folder name from the path
                roi_folder_name = Path(folder).parts[1]

                #Makes sure the output folder name exists for this ROI
                Path(join(processed_output_dir, roi_folder_name)).mkdir(exist_ok=True) 

                #The output file is named the same as the input file
                save_path = join(processed_output_dir, roi_folder_name, img_file_name)      

                #Save the denoised file
                tp.imsave(save_path,Img_DIMR_DeepSNF.astype('float32'))

            completed_channels.append(channel_name)
        except Exception as e:

            print(f"Error in channel {channel_name}: {Exception}: {e}")
            error_channels.append(f"{channel_name}: {Exception}: {e}")

    print("Successfull with channels:")
    print(completed_channels)
    print("Channels with errors:")
    print(error_channels)
    deep_SNF_batch.completed = completed_channels
    deep_SNF_batch.errors = error_channels
    

# This function copies all the original tiffs into a new folder, then copies over the processed ones, giving a new folder with all the right images that match up with the original ometiffs    
    
def combine(raw_directory = "tiffs",
            processed_output_dir = "processed",
            combined_dir="combined"):
    
    # Create folders
    combined_dir_create = Path(combined_dir)
    combined_dir_create.mkdir(exist_ok=True)
    
    # Copy raw tiffs into new directory
    print(f'Copying original files from: {raw_directory}...')
    distutils.dir_util.copy_tree(raw_directory, combined_dir)
    
    # Copy processed tiffs over, hopefully overwriting
    print(f'Adding in processed files from: {processed_output_dir}...')
    distutils.dir_util.copy_tree(processed_output_dir, combined_dir)
    
# Side by side comparisson of before and after processing

def qc_check_side_by_side(channels=[],
                          process_all_channels=False,
                            colourmap ='jet',
                            dpi=200,
                            save=True,
                            save_dir='qc_images',
                            do_all_channels=True,
                            hide_images=True,
                            raw_directory='tiffs',
                            processed_output_dir='processed',
                            quiet=True):
    
    if not isinstance(channels, list):
        channels=[channels]    
    
    if process_all_channels:
        channels = channel_df['channel'].tolist()
    
    # Create folders
    save_dir = Path(save_dir)
    save_dir.mkdir(exist_ok=True)

    if do_all_channels:
        channel_list=all_data_channels
    else:
        channel_list=channels

    # Error catching lists
    error_channels=[]
    completed_channels=[]    

    for channel_name in channel_list:

        try:

            raw_Img_collect, raw_Img_file_list, raw_img_folders = load_imgs_from_directory(raw_directory, channel_name,quiet=quiet)
            pro_Img_collect, pro_Img_file_list, pro_img_folders = load_imgs_from_directory(processed_output_dir, channel_name,quiet=quiet)

            fig, axs = plt.subplots(len(raw_Img_collect), 2, figsize=(10, 5*len(raw_Img_collect)), dpi=dpi)

            count = 0
            for r_img,p_img,r_img_name in zip(raw_Img_collect,pro_Img_collect,raw_Img_file_list):
                im1= axs.flat[count].imshow(r_img, vmin = 0, vmax = 0.5*np.max(r_img), cmap = colourmap)
                divider = make_axes_locatable(axs.flat[count])
                cax = divider.append_axes('right', size='5%', pad=0.05)
                fig.colorbar(im1, cax=cax, orientation='vertical')
                axs.flat[count].set_ylabel(str(r_img_name))
                count=count+1

                im2 = axs.flat[count].imshow(p_img, vmin = 0, vmax = 0.5*np.max(p_img), cmap = colourmap)
                divider = make_axes_locatable(axs.flat[count])
                cax = divider.append_axes('right', size='5%', pad=0.05)
                fig.colorbar(im2, cax=cax, orientation='vertical')    
                count=count+1 

            fig.savefig(join(save_dir, channel_name+'.png'))

            if hide_images:
                plt.close()

            completed_channels.append(channel_name)

        except Exception as e:

            print(f"Error in channel {channel_name}: {Exception}: {e}")
            error_channels.append(f"{channel_name}: {Exception}: {e}")

    print("Successfull with channels:")
    print(completed_channels)
    print("Channels with errors:")
    print(error_channels)
    qc_check_side_by_side.completed = completed_channels
    qc_check_side_by_side.errors = error_channels        
    
    
# Function to reassemble TIFF stacks - assumes they are in the right order, which by default they should be if 'unstack' is used

def reassemble_stacks(restack_input_folder = 'combined',
                      restacked_output_folder = 'tiffs_restacked',
                      save_panel=True, #Will save a csv file that details each chanel in the new stack
                      re_order=None,#Give a list of the file names in their correct order
                      ascending_sort_names=True): #If not reordering, this will force a sort of the file names into ascending order
    
    global file_df
        
    # Make output directories if they don't exisit
    restack_input_folder = Path(restack_input_folder)
    output = Path(restacked_output_folder)
    output.mkdir(exist_ok=True)

    
    # Get a list of paths of ROI folder
    Img_folders = glob(join(restack_input_folder, "*", ""))

    print('Savings stacks...')
    for i in tqdm(Img_folders):

        tiff_files = list(Path(i).rglob('*.tiff'))
        file_names = [os.path.splitext(os.path.splitext(os.path.basename(tiff_files[x]))[0])[0] for x in range(len(tiff_files))]
        file_df = pd.DataFrame(zip(file_names,tiff_files),columns=['File name','Path']).set_index('File name')
        
        if re_order:
            file_df=file_df.reindex(re_order)
        elif ascending_sort_names:
            file_df=file_df.sort_index(ascending=True)            
        
        image_stack=[]

        for file in file_df['Path']:
            im = tp.imread(str(file)).astype('float32')
            image_stack.append(im)

        image_stack = np.asarray(image_stack)

        save_path=join(restacked_output_folder, Path(i).parts[1]+".tiff")

        tp.imsave(save_path,image_stack.astype('float32'))
        
        if save_panel:
            panel_path =join(restacked_output_folder, Path(i).parts[1]+".csv")
            file_df.to_csv(panel_path)

def gpu_test():
    import tensorflow as tf

    if tf.test.is_built_with_cuda()==True:
        print('GPU accceleration enabled \n')
        print(tf.config.list_physical_devices('GPU'))
    else:
        print('GPU not found! Check TensorFlow and CUDA setup')

def reassemble_stacks2(channels = [], 
                       restack_input_folder = 'combined',
                       restacked_output_folder = 'Deep_Cell',
                       process_all_channels=False,
                       do_all_channels=True): 
    
    global file_df
    
    # Make output directories if they don't exisit
    restack_input_folder = Path(restack_input_folder)
    output = Path(restacked_output_folder)
    output.mkdir(exist_ok=True)
    
    if not isinstance(channels, list):
        channels=[channels]    
    
    if process_all_channels:
        channels = channel_df['channel'].tolist()
    
    if do_all_channels:
        channel_list=all_data_channels
    else:
        channel_list=channels
        
    n = len(channels)
    print(f'\nStacking the following {n} channels... \n')
    print(channels)

    # Get a list of paths of ROI folder
    Img_folders = glob(join(restack_input_folder, "*", ""))
        
    for channels in channel_list:
        
        try:
            
            for i in tqdm(Img_folders):

                tiff_files = list(Path(i).rglob('*.tiff'))
                file_names = [os.path.splitext(os.path.splitext(os.path.basename(tiff_files[x]))[0])[0] for x in range(len(tiff_files))]
                file_df = pd.DataFrame(zip(file_names,tiff_files),columns=['File name','Path']).set_index('File name')
        
            image_stack=[]

            for file in file_df['Path']:
                im = tp.imread(str(file)).astype('float32')
                image_stack.append(im)

            image_stack = np.asarray(image_stack)

            save_path=join(restacked_output_folder, Path(i).parts[1]+".tiff")

            tp.imsave(save_path,image_stack.astype('float32'))
        
        except Exception as e:

            print(f"Error in channel {channel_name}: {Exception}: {e}")
            error_channels.append(f"{channel_name}: {Exception}: {e}")
            

In [ ]:
#save the session
dill.dump_session('IMC_Denoise_COVID_Malawi.db')

In [42]:
#load the session
dill.load_session('IMC_Denoise_COVID_Malawi.db')

# GPU Test

This should return 'True' and the name of your GPU. If it doesn't, something has gone wrong in the setup of TensorFlow and/or CUDA that allows GPU-acceleration. Without it, the script will run incredibly low

In [ ]:
gpu_test()

# 1. Unpack tiff stacks

A- input_folder = The folder where the stacked tiff files are. You should be able to just copy and paste the whole .ome.tiff folder that the Bodenmiller pipeline creates after it has extracted the tiff files from the MCD files. This folder also contains the .csv panel files, copy those too! Any panorama files will also be in the same folder, but they won't be used here.
B- unstacked_output_folder = Where the 'unstacked' tiff files will be stored. They will be unpacked into a single folder per ROI.
C- use_panel_files = If you are using the Bodenmiller pipeline, leave this as True. It will use the .csv panel files for each ROI to properly label the unpacked channels with their metal tags and antigen targets, and will create a file called ROI_data.csv which will store all the information

In [ ]:
unstack_tiffs() 

By default, 'all_channels' will be all the channels with names

In [99]:
all_channels=channel_df['channel'].tolist()
all_channels

['Y89_Sma',
 'In113_Cd68',
 'In115_Cd235ab',
 'La139_Pan-cytokeratin',
 'Pr141_Cd38',
 'Nd142_MHC1',
 'Nd143_Vimentin',
 'Nd144_CD14',
 'Nd145_ICAM1',
 'Nd146_CD16',
 'Sm147_iNOS',
 'Nd148_CD66b',
 'Sm149_CD11b',
 'Nd150_Cd44',
 'Eu151_Cd107a',
 'Sm152_Cd45',
 'Eu153_Cd31',
 'Sm154_CD11c',
 'Gd155_Foxp3',
 'Gd156_Cd4',
 'Gd158_SARS-Cov2',
 'Tb159_vWF',
 'Gd160_Vista',
 'Dy161_Cd20',
 'Dy162_Cd8a',
 'Dy163_Iba1',
 'Dy164_Arginase1',
 'Ho165_Fibrinogen',
 'Er166_Cd74',
 'Er167_GranzymeB',
 'Er168_Ki67',
 'Tm169_Collagen1',
 'Er170_Cd3',
 'Yb171_pERK',
 'Yb172_ClvdCaspase3',
 'Yb173_Cd45RO',
 'Yb174_MHC2',
 'Lu175_CD206',
 'Ir191_DNA1',
 'Ir193_DNA3',
 'Pt196_Cd163']

# 2. QC check on raw data

This does some QC checks on the raw data, and isn't strictly necessary for later analyses. By default it won't be displayed, but will be saved to a new'qc_images' directory.

In [ ]:
#See the function in first box or run this for the default options
qc_heatmap(channels=all_channels,
          hide_figures=True) #This will not display all the images

# 3. Run DeepSNF training and image denoising

channels = Specify which channels to process here, e.g. if you only want to process a couple. When you define channels, it will search through the source directories for any images matching that name. Therefore, be careful with channels with overlapping names! Eg. CD4 and CD45!
raw_directory = This should be the same as 'unstacked_output_folder' above (by default is) - where the unstacked images were stored, with each ROI being a folder containing all its images.
processed_output_dir = The folder where the processed images will be stored. They will be in the same format as above - each ROI its own folder containing all its images.
Deep SNF settings
These all have accompanying explanations, and can mostly be left alone. Ones you may want to change include...
train_batch_size If you are getting 'out of memory' errors you may need to reduce this to work on a GPU (e.g. to 32), or increase if you have a very good GPU setup.
patch_step_size This is the frequency (in pixels) at which patches are taken from the dataset for training. If you are getting errors of being out of memory, usually because you have a huge dataset, increase this from its default of 60, to 100-150. Also, if you have a channel with very few cells or sparse taining, you may also want to decrease this a lot, potentially to <50

In [ ]:
deep_SNF_batch(channels=["Ir193_DNA3"],  #This will do all channels, as we defined above, but you can just give a list of channels
               patch_step_size=80,
               train_batch_size=128)

# 4. Side-by-side comparisson to check performance of denoising¶

This will do a side-by-side comparisson for each ROI for before and after denoising, for each channel, then save the image to the 'qc_images' directory. By default, it will look for the raw and processed images in the directories specified above, but you can point to specific directories insteead (raw_directory and processed_output_dir)

In [ ]:
qc_check_side_by_side(channels=all_channels, 
                      dpi=100), #By default ever ROI is done side-by-side, which can make images huge!

# 5. QC check on processed data

This will create heatmaps and PCAs that can be compared with those generated in step 2 on the raw data

In [ ]:
qc_heatmap(directory='processed', normalize='max', channels=all_channels, hide_figures=True)
qc_heatmap(directory='processed', normalize='zscore', channels=all_channels, hide_figures=True)

# 6. Combine
Create a new folder which will contain all the newly processed tiffs, and the original tiffs which were not processed. **If you do not want to use all the newly processed tiffs, then you will have to do this step manually!**

In [ ]:
combine()

In [ ]:
#not working
#transfer(channels= "Er166", process_all_channels=False, do_all_channels=False,
           # raw_directory = "tiffs",
           # processed_output_dir = "processed2")

# 7. Reassemble TIFF stacks

At this point, we want to reassemble the invidiual images back into stacks so we can put them back into the Bodenmiller pipeline, replacing the ones originally generated. You may want to keep backups of the unprocessed tiffs!
By default, this pipeline will use all the processed image! If you only want to use some of the images, then manually assemble the individual TIFFs in the folders ready to be restacked
restack_input_folder = This should be the same as processed output directory above - where the processed images were stored, with each ROI being a folder containing all its images. Default is 'tiffs'.
restack_input_folder = Where the processed and now restacked images should be place. Default is 'tiffs_restacked'.

In [ ]:
reassemble_stacks(restack_input_folder = 'combined',
                  restacked_output_folder = 'Deep_Cell',                  
                  save_panel=True, 
                  re_order=None)

In [112]:
reassemble_stacks2(channels = ["Y89_Sma", "In113_Cd68"],
                   restack_input_folder = 'combined',
                   restacked_output_folder = 'Deep_Cell',
                   process_all_channels=False,
                   do_all_channels=True) 


Stacking the following 2 channels... 

['Y89_Sma', 'In113_Cd68']


100%|███████████████████████████████████████| 131/131 [00:00<00:00, 1476.61it/s]
/var/folders/0m/4rwt_z816mlcps48xsc34jvc0000gn/T/ipykernel_8123/1548659390.py:559: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tp.imsave(save_path,image_stack.astype('float32'))
100%|███████████████████████████████████████| 131/131 [00:00<00:00, 2154.88it/s]
/var/folders/0m/4rwt_z816mlcps48xsc34jvc0000gn/T/ipykernel_8123/1548659390.py:559: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tp.imsave(save_path,image_stack.astype('float32'))
100%|███████████████████████████████████████| 131/131 [00:00<00:00, 2014.92it/s]
/var/folders/0m/4rwt_z816mlcps48xsc34jvc0000gn/T/ipykernel_8123/1548659390.py:559: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tp.imsave(save_path,image_stack.astype('float32'))
100%|███████████████████████████████████████| 131/131 [00:00<00:00, 2174.92it/s]
/var/folders/0m/4rwt_z816mlcps48xsc34jvc0000gn

100%|███████████████████████████████████████| 131/131 [00:00<00:00, 2067.88it/s]
/var/folders/0m/4rwt_z816mlcps48xsc34jvc0000gn/T/ipykernel_8123/1548659390.py:559: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tp.imsave(save_path,image_stack.astype('float32'))
100%|███████████████████████████████████████| 131/131 [00:00<00:00, 2068.75it/s]
/var/folders/0m/4rwt_z816mlcps48xsc34jvc0000gn/T/ipykernel_8123/1548659390.py:559: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tp.imsave(save_path,image_stack.astype('float32'))
100%|███████████████████████████████████████| 131/131 [00:00<00:00, 2139.33it/s]
/var/folders/0m/4rwt_z816mlcps48xsc34jvc0000gn/T/ipykernel_8123/1548659390.py:559: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tp.imsave(save_path,image_stack.astype('float32'))
100%|███████████████████████████████████████| 131/131 [00:00<00:00, 2158.15it/s]
/var/folders/0m/4rwt_z816mlcps48xsc34jvc0000gn